In [1]:
import numpy as np
import tensorflow as tf

In [2]:
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {v: i for i, v in enumerate(char_arr)}
dic_len = len(num_dic)

In [3]:
seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'],
            ['girl', '소녀'], ['kiss', '키스'], ['love', '사랑']]

In [4]:
def make_batch(seq_data):
    input_batch, output_batch, target_batch = [], [], []
    
    for seq in seq_data:
        inpt = [num_dic[w] for w in seq[0]]
        output = [num_dic[w] for w in ('S' + seq[1])]
        target = [num_dic[w] for w  in (seq[1] + 'E')]
        # Each number from num_dic becomes an one-hot vector
        input_batch.append(np.eye(dic_len)[inpt]) # Create identity matrix
        output_batch.append(np.eye(dic_len)[output]) # Create identity matrix 
        target_batch.append(target)
    return input_batch, output_batch, target_batch

In [5]:
learning_rate = 0.01
n_hidden = 128 # Number of hidden states
total_epoch = 100
n_input = n_class = dic_len # Number of characters

In [6]:
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

In [7]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

In [8]:
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    
    outputs, dec_states = tf.nn.dynamic_rnn(
        dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32
    )

In [9]:
model = tf.layers.dense(outputs, n_class, activation=None)

cost = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets)
)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    print('Epoch: {:03}, cost = {:.6f}'.format(epoch + 1, loss))

Epoch: 001, cost = 3.703137
Epoch: 002, cost = 2.751431
Epoch: 003, cost = 1.714648
Epoch: 004, cost = 0.829796
Epoch: 005, cost = 0.826042
Epoch: 006, cost = 0.396648
Epoch: 007, cost = 0.289734
Epoch: 008, cost = 0.195465
Epoch: 009, cost = 0.155327
Epoch: 010, cost = 0.085273
Epoch: 011, cost = 0.067709
Epoch: 012, cost = 0.051858
Epoch: 013, cost = 0.057874
Epoch: 014, cost = 0.021420
Epoch: 015, cost = 0.034621
Epoch: 016, cost = 0.028905
Epoch: 017, cost = 0.005745
Epoch: 018, cost = 0.007259
Epoch: 019, cost = 0.013755
Epoch: 020, cost = 0.014086
Epoch: 021, cost = 0.022612
Epoch: 022, cost = 0.002314
Epoch: 023, cost = 0.003651
Epoch: 024, cost = 0.035566
Epoch: 025, cost = 0.001597
Epoch: 026, cost = 0.006961
Epoch: 027, cost = 0.003130
Epoch: 028, cost = 0.002283
Epoch: 029, cost = 0.003220
Epoch: 030, cost = 0.004545
Epoch: 031, cost = 0.000992
Epoch: 032, cost = 0.002446
Epoch: 033, cost = 0.002930
Epoch: 034, cost = 0.001522
Epoch: 035, cost = 0.000964
Epoch: 036, cost = 0

In [11]:
def translate(word):
    seq_data = [word, 'P' * len(word)]
    i_batch, o_batch, t_batch = make_batch([seq_data])
    
    prediction = tf.argmax(model, 2)
    result = sess.run(prediction,
                      feed_dict={enc_input: i_batch,
                                 dec_input: o_batch,
                                 targets: t_batch})
    
    decoded = [char_arr[i] for i in result[0]]
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    return translated

In [12]:
print('word -> ', translate('word'))
print('wodr -> ', translate('wodr'))
print('love -> ', translate('love'))
print('loev -> ', translate('loev'))
print('abcd -> ', translate('abcd'))

word ->  단어
wodr ->  단어
love ->  사랑
loev ->  사랑
abcd ->  사랑


Warning: Having a small toy dataset will only provide low accuracy results.